> **DO NOT EDIT IF INSIDE `annadl_f19` folder** 

# Assignment 2

**Handin in Peergrade**: *Friday*, November 15, 2019, 23:59<br>
**Peergrading deadline**: *Monday*, November 18, 2019, 23:59<br>
**Peergrading feedback deadline**: *Friday*, November 22, 2019, 23:59

[**Feedback**](http://ulfaslak.com/vent)

## Problems

### Week 5

In [ ]:
%matplotlib inline

import numpy as np
import random
import sys
import io
import requests as rq
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop

In [ ]:
response = rq.get("http://www.dailyscript.com/scripts/pulp_fiction.html")
text = BeautifulSoup(response.content, "html.parser").getText()

> **Ex. 5.1.4**: Did the network finish training? Consider the generated text across epochs.
1. In the early batches (0-10), the generated text looks very bad. Can you explain why the low diversity generated text contains almost only the symbol " " (that is, spaces)?
2. The high diversity generated text is messed up too, but in a different way. Explain how.
3. In later batches (20-30) what do you notice is off about the low diversity generated text?

In [ ]:
# Q1: What is the purpose of this block? When is `char_indices` used? What about `indices_char`?
# Creates a sorted list of characters in the text
chars = sorted(list(set(text)))
# Creates a dict with key character and value the index of every instance of the character
char_indices = dict((c, i) for i, c in enumerate(chars))
# Creates a dict with key index and value the character at that index
indices_char = dict((i, c) for i, c in enumerate(chars))

# Q2: What is the purpose of this block? What does the `seqlen` and `step` parameters do?
# seqlen is used as a counter to evaluate the text in 40 character blocks
seqlen = 40
step = seqlen
sentences = []
for i in range(0, len(text) - seqlen - 1, step):
    sentences.append(text[i: i + seqlen + 1])

# Q3: What about this block? What is `x` and what is `y`? Why do they have this dimensionality?
# 'x' and 'y' are empty arrays of 40 characters 
x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    # Q3a: What happens in this loop?
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1


# def create_model():
#     return tf.keras.models.Sequential([
#         tf.keras.layers.LSTM(128, input_shape=(seqlen,len(chars)), return_sequences = True),
#         tf.keras.layers.Dense(len(chars), activation='softmax')])

# model = create_model()
        
        
# Q4: Here we build the model. What does the `return_sequences` argument do? Why the dense layer at the end?
# Return sequences returns the sequences 
model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - seqlen - 1)
    
    # Q5: What does diversity do?
    # Diversity is used to sample from several different values for the next index
    for diversity in [0.2, 0.5, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + seqlen]
        # sentence = 'adsfghnsdRTYJHfhilfjrtkyslkjhfgaEDFGDSIA'
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            
            # What is the dimensionality of `preds`? Why do we input `preds[0, -1]` to the `sample` function?
            # The input `preds[0, -1]` 
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=50,
          callbacks=[print_callback])

1. The low diversity generated text in the early batches is mostly spaces. This suggests that the RNN is recognizes the spacing between words and predicts that there will be more spaces, generating text based off of that.
2. The high diversity generated text in early batches contains jumbles of letters separated by spaces, which seems to suggest that the RNN recognizes a pattern of spacing between words and a pattern of words as groups of letters. In later batches, the high diversity generated text is occasionally less meaningful than the low diversity text as the output just becomes gibberish with made-up words. 
3. In the later batches, the low diversity generated text seems to repeat the same words in the output multiple times.

> **Ex. 5.1.6**: Do the same as above, but for 40 random letters (e.g. smash away on your keyboard) as seed. What happens? Can you explain why?

The text generated using the 40 random characters tends to contain more nonsense words when compared to Ex. 5.1.4 which used existing text in the script as the seed. 

### Week 6

> **Ex. 6.1.1**: In your own words, explain what the following function arguments do in
the different model loading functions:
1. `include_top`
1. `weights`
1. `input_shape`
1. `pooling`
1. `classes`
1. Explain what 'global pooling' does, and why it is needed when `include_top=False`

> **Ex. 6.1.2**: Following Jason's example under 'Pre-Trained Model as Classifier'
classify [this image](https://66.media.tumblr.com/tumblr_mc46e7Zm4R1qbqngeo1_1280.jpg).
Print not just the most likely label, but everything that `decode_predictions` returns.
>
> ***Important***: *Don't use VGG as he does. It's 500 MB to download, and will take too long.
> Use one of the smaller models instead ([here](https://keras.io/applications/#documentation-for-individual-models)'s an overview of model sizes), such as DenseNet121.*

### Week 7

> **Ex. 7.1.1**: What is typically the input and output of an autoencoder? What loss function can be used?

> **Ex. 7.1.3**: Purely in terms of architecture, what is the difference between an autoencoder and a variational autoencoder (VAE)?

> **Ex. 7.1.4**: Regular autoencoders are trained to minimize a loss function with no regard to how the latent space is organized. Therefore, continuity is not guaranteed and similar datapoints may not be close to each other. We can thus say that the network is overfitting, because it uses any organization of training points in this space to minimize the loss, and is, therefore, not likely to work well with unseen data. VAEs are a regularized form of autoencoders, invented to solve this problem. Importantly, they guarantee that similar points are close in the latent space. How do they achieve this?
    > * How are datapoints represented in the VAE latent space? What is the intuition behind this?
    > * How is the loss function different? What is the purpose of the second term (the KL divergence)?
>
> *Hint: Check out this [blog post](https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73) and read the section "Intuitions about the regularisation"*

> **Ex. 7.2.1**: Explain in your own words how the GAN works. Touch upon:
    > * What do the generator and discriminator networks do?
    > * What are their respective input and output?
    > * What would the accuracy of the discriminator be, faced with a perfect generator?

> **Ex. 7.2.4**: How do you transform one image to another using backprop and
gradient descent? Why does this not always work that well? How is transfer learning
used to make it work?